In [1]:
import openrouteservice as ors
import folium
import folium.plugins as plugins
import random
import pandas as pd

client = ors.Client(key='5b3ce3597851110001cf6248facc383c642b420fa9818331b313169e')

In [2]:
def visualizeTSP(readLocation, readLocationTSP, writeFolder, writeLocationTSP):
    print(readLocationTSP)
    truckRoute = pd.read_csv(readLocationTSP + "\\TSPTruckRoute.csv")
    nodes = pd.read_csv(readLocation + "\\Nodes.csv")
    
    m = folium.Map(location=[40.80533085, 29.442315], tiles='cartodbpositron', zoom_start=13)

    # Add Nodes
    for index, row in nodes.iterrows():
        color = None
        serveToolTip = None
        if(row["Type"] == "UAVELIGIBLE"):
            color = "orange"
        else:
            color = "cyan"
        if(index == 0 or index == nodes.shape[0]-1):
            icon = plugins.BeautifyIcon(icon = "truck", iconShape="marker", backgroundColor="white")
        else:
            icon = plugins.BeautifyIcon(icon = "", iconShape="marker", number = row["Index"], backgroundColor=color)
        folium.Marker([row["Y"], row["X"]], icon=icon, tooltip=serveToolTip).add_to(m)
    
    
    # Add Truck Routes
    for i in range(0, truckRoute.shape[0] - 1):
        popup = "{i} --> {j}"

        coordinates = [[truckRoute["X"][i], truckRoute["Y"][i]], [truckRoute["X"][i+1], truckRoute["Y"][i+1]]]
        route = client.directions(
        coordinates=coordinates,
        profile='driving-car',
        format='geojson',
        validate=False,)
        plugins.AntPath(locations=[list(reversed(coord)) 
                               for coord in 
                               route['features'][0]['geometry']['coordinates']], 
                               tooltip = popup.format(i = truckRoute["Index"][i], j = truckRoute["Index"][i+1]), linecap ="round").add_to(m)
    
    m.save(writeLocationTSP)

In [3]:
def visualize(readLocation, writeFolder, writeLocation):
    # Read Data
    print(readLocation)
    nodes = pd.read_csv(readLocation + "\\Nodes.csv")
    truckRoute = pd.read_csv(readLocation + "\\TruckRoute.csv")
    droneRoute = pd.read_csv(readLocation + "\\DroneRoute.csv")
    
    m = folium.Map(location=[40.80533085, 29.442315], tiles='cartodbpositron', zoom_start=13)

    # Add Nodes
    for index, row in nodes.iterrows():
        color = None
        serveToolTip = None
        if(row["Type"] == "UAVELIGIBLE"):
            color = "orange"
        else:
            color = "cyan"
        if(index == 0 or index == nodes.shape[0]-1):
            icon = plugins.BeautifyIcon(icon = "truck", iconShape="marker", backgroundColor="white")
        else:
            for indexDrone, rowDrone in droneRoute.iterrows():
                if(row["Index"] == rowDrone["jIndex"]):
                    serveToolTip = "Served by a drone at time {t}".format(t = rowDrone["Launch[j]"])
            for indexTruck, rowTruck in truckRoute.iterrows():
                if(row["Index"] == rowTruck["Index"]):
                    serveToolTip = "Served by a truck at time {t}".format(t = rowTruck["CompletionTime"])
                
            icon = plugins.BeautifyIcon(icon = "", iconShape="marker", number = row["Index"], backgroundColor=color)
        folium.Marker([row["Y"], row["X"]], icon=icon, tooltip=serveToolTip).add_to(m)
    
    
    # Add Truck Routes
    for i in range(0, truckRoute.shape[0] - 1):
        popup = "{i} --> {j}"

        coordinates = [[truckRoute["X"][i], truckRoute["Y"][i]], [truckRoute["X"][i+1], truckRoute["Y"][i+1]]]
        route = client.directions(
        coordinates=coordinates,
        profile='driving-car',
        format='geojson',
        validate=False,)
        plugins.AntPath(locations=[list(reversed(coord)) 
                               for coord in 
                               route['features'][0]['geometry']['coordinates']], 
                               tooltip = popup.format(i = truckRoute["Index"][i], j = truckRoute["Index"][i+1]), linecap ="round").add_to(m)

    # Add Drone Routes
    colorsForDroneRoutes = ["red", "purple", "yellow", "green", "orange"]

    for rowNumber in range(0, droneRoute.shape[0]):
        i = (droneRoute["iY"][rowNumber], droneRoute["iX"][rowNumber])
        j = (droneRoute["jY"][rowNumber], droneRoute["jX"][rowNumber])
        k = (droneRoute["kY"][rowNumber], droneRoute["kX"][rowNumber])

        color = colorsForDroneRoutes[droneRoute["DroneIndex"][rowNumber]]

        ijRouteToolTip = "{i} --> {j} | LaunchTime[{i}]: {launch} / ArrivalTime[{j}]: {arrival}"
        jkRouteToolTip = "{j} --> {k} | LaunchTime[{j}]: {launch} / RetrievalTime[{k}]: {retrieval}"
        plugins.AntPath(locations = [i,j], color=color, tooltip = ijRouteToolTip.format(i = droneRoute["iIndex"][rowNumber],
                                                                                       j = droneRoute["jIndex"][rowNumber],
                                                                                       launch = droneRoute["Launch[i]"][rowNumber],
                                                                                       arrival = droneRoute["Arrival[j]"][rowNumber])).add_to(m)
        plugins.AntPath(locations = [j,k], color=color, tooltip = jkRouteToolTip.format(j = droneRoute["jIndex"][rowNumber],
                                                                                       k = droneRoute["kIndex"][rowNumber],
                                                                                       launch = droneRoute["Launch[j]"][rowNumber],
                                                                                       retrieval = droneRoute["Retrieval[k]"][rowNumber])).add_to(m)
    #Save visualization
    m.save(writeLocation)

In [4]:
nArray = [8]
iTimes = [1, 2, 3, 4, 5]
drones = [5]
penaltyMins = [40]

for n in nArray:
    for i in iTimes:
        for d in drones:
            for minute in penaltyMins:
                readLocationTSP = "..\\NewModel\\Data_{n}_{d}_{i}\\Penalty_{minute}Mins\\TSP"
                readLocationTSP = readLocationTSP.format(n = n, d = d, i = i, minute = minute)
                
                readLocation = "..\\NewModel\\Data_{n}_{d}_{i}\\Penalty_{minute}Mins"
                readLocation = readLocation.format(n = n, d = d, i = i, minute = minute)

                writeFolder = "..\\NewModel\\Data_{n}_{d}_{i}\\Penalty_{minute}Mins"
                writeFolder = writeFolder.format(n = n, d = d, i = i, minute = minute)

                writeLocation = "..\\NewModel\\Data_{n}_{d}_{i}\\Penalty_{minute}Mins\\map.html"
                writeLocation = writeLocation.format(n = n, d = d,i = i, minute = minute)
                
                writeLocationTSP = "..\\NewModel\\Data_{n}_{d}_{i}\\Penalty_{minute}Mins\\TSP\\map.html"
                writeLocationTSP = writeLocationTSP.format(n = n, d = d,i = i, minute = minute)

                visualize(readLocation, writeFolder, writeLocation)
                visualizeTSP(readLocation, readLocationTSP, writeFolder, writeLocationTSP)


..\NewModel\Data_8_5_1\Penalty_40Mins
..\NewModel\Data_8_5_1\Penalty_40Mins\TSP
..\NewModel\Data_8_5_2\Penalty_40Mins
..\NewModel\Data_8_5_2\Penalty_40Mins\TSP
..\NewModel\Data_8_5_3\Penalty_40Mins
..\NewModel\Data_8_5_3\Penalty_40Mins\TSP
..\NewModel\Data_8_5_4\Penalty_40Mins
..\NewModel\Data_8_5_4\Penalty_40Mins\TSP


c:\users\ulvif\appdata\local\programs\python\python38\lib\site-packages\openrouteservice\client.py:211: UserWarning: Rate limit exceeded. Retrying for the 1st time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
c:\users\ulvif\appdata\local\programs\python\python38\lib\site-packages\openrouteservice\client.py:211: UserWarning: Rate limit exceeded. Retrying for the 2nd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
c:\users\ulvif\appdata\local\programs\python\python38\lib\site-packages\openrouteservice\client.py:211: UserWarning: Rate limit exceeded. Retrying for the 3rd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
c:\users\ulvif\appdata\local\programs\python\python38\lib\site-packages\openrouteservice\client.py:211: UserWarning: Rate limit exceeded. Retrying for the 4th time.
  warnings.warn('Rate limit exceeded. Retrying for the 

..\NewModel\Data_8_5_5\Penalty_40Mins
..\NewModel\Data_8_5_5\Penalty_40Mins\TSP
